# Performance Evaluation Notebook

## Import libraries

In [1]:
from collections import namedtuple
import os
import pandas as pd
from models.languageModels.translationModels import *
from models.learningModels import learningKerasModels
from utils.utils import *
from utils import generators
import tensorflow.keras as keras
from tensorflow.keras import optimizers
from collections import defaultdict
import tensorflow as tf
from utils import jiwer
#from utils import evaluation
from utils import bleu
from utils import rouge
import main
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
import nltk
import copy
nltk.download('wordnet')

Using TensorFlow backend.
[nltk_data] Downloading package wordnet to /home/jota/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Computing performance metrics

In [2]:
# Translation file to evaluate
results = open(os.getcwd()+'/results/translations/EXTRA2-PHOENIX-RGB-LSTM-TOP-128-60.pkl',"rb")
results = pickle.load(results)

def metrics_on_corpus(result):
    annotations ='../Datasets/Pre-procesados/SLR/RWTH-Phoenix/phoenixT/phoenix-2014-T.v3/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/annotations/'
    ann = pd.read_csv(annotations+'annotations.csv', delimiter='|')
    
    for data in result.keys():
        references = []
        translations = []
        references_rouge = []
        translations_rouge = []
        wert = 0.0
        meteort = 0.0
        for c, name in enumerate(result[data].keys()):
            translation = result[data][name][0]
            videoName = name.split('.')[0]#[:11]
            reference = ann['Translation'][ann['Video Name']==videoName].values[0].lower()
            if len(translation)==1:
                flag=True
            else:
                flag=False
            if '<eos>' in translation:
                translation.remove('<eos>')
            translation = " ".join(translation)
            wert = wert + jiwer.wer(truth = reference, hypothesis = translation) 
            meteort = meteort + single_meteor_score(reference, translation)
            
            translations.append(translation.split(" "))
            translations_rouge.append(translation)
            references.append([reference.split(" ")])
            references_rouge.append(reference)
            
        print(len(references))
        rouge_score_map = rouge.rouge(translations_rouge, references_rouge)
        print(data, 'rouge', 100 * rouge_score_map["rouge_l/f_score"])
        print(data , 'WER', (wert/len(references))*100)
        print(data , 'Meteor', (meteort/len(references))*100)
        for max_ in range(1,5):
            bleu_score, _, _, _, _, _ = bleu.compute_bleu(references, translations, max_order=max_)
            print(data, 'bleu',max_,bleu_score*100)
        
            
metrics_on_corpus(results)

7096
train rouge 12.382089768228814
train WER 111.61264240565383
train Meteor 9.605041925211696
train bleu 1 14.540281609910316
train bleu 2 6.655631857839774
train bleu 3 3.864429248433179
train bleu 4 2.6301798126849016
642
test rouge 11.532112772508134
test WER 116.68140808412608
test Meteor 9.013053143051897
test bleu 1 14.098855489875486
test bleu 2 6.6608072558548965
test bleu 3 4.144581078293574
test bleu 4 2.993728489341203
519
dev rouge 12.36314307351333
dev WER 110.63273458307694
dev Meteor 9.506925402137815
dev bleu 1 14.741156566901905
dev bleu 2 6.710022673659551
dev bleu 3 4.036408024659126
dev bleu 4 2.872527497369305
